In [2]:
import gym
import numpy as np
from stable_baselines3.ddpg.policies import MultiInputPolicy
from stable_baselines3.common.noise import OrnsteinUhlenbeckActionNoise, NormalActionNoise
from stable_baselines3 import DDPG, TD3, SAC, PPO, HerReplayBuffer

Create the environmnet

In [3]:
env = gym.make('FetchReach-v1')

Create the model for all the algorithms

In [7]:
#model of the class

model_class_ddpg = DDPG  # works with SAC, DDPG and TD3
model_class_sac = SAC
model_class_td3 = TD3

#number of timesteps
n_timesteps = 300000

#log interval
log_interval = 4

#tensorboar log
log="./fetchreachHerGraphs/"
# Available strategies (cf paper): future, final, episode
goal_selection_strategy = 'future' # equivalent to GoalSelectionStrategy.FUTURE

# If True the HER transitions will get sampled online
online_sampling = True

In [ ]:
# Initialize the model
model_ddpg = model_class_ddpg(
    "MultiInputPolicy",
    env,tensorboard_log=log,
    replay_buffer_class=HerReplayBuffer,
    # Parameters for HER
    replay_buffer_kwargs=dict(
        n_sampled_goal=4,
        goal_selection_strategy=goal_selection_strategy,
        online_sampling=online_sampling
    ),
    verbose=1,
)
# Train the model and then save it
model_ddpg.learn(total_timesteps=n_timesteps, log_interval=log_interval)

model_ddpg.save("ddpg_fetch_reach_her")


In [8]:
"""
The hyperparameters are the ones from https://github.com/DLR-RM/rl-baselines3-zoo/blob/master/hyperparams/sac.yml
They have been optimized by means of Optuna.
"""
model_sac = model_class_sac("MultiInputPolicy",
            env,
            learning_rate=0.001,
            learning_starts = 1000,
            gamma = 0.95,
            replay_buffer_class=HerReplayBuffer,
            # Parameters for HER
            replay_buffer_kwargs=dict(
                n_sampled_goal=4,
                goal_selection_strategy='future',
                online_sampling=True,
                ),
            tensorboard_log=log,
            policy_kwargs=dict(net_arch=[64, 64]),
            verbose=1)

model_sac.learn(total_timesteps=n_timesteps, log_interval=log_interval)

model_sac.save("sac_fetch_reach_her")
#######################################
model_td3 = model_class_td3(
    "MultiInputPolicy",
    env,tensorboard_log=log,
    replay_buffer_class=HerReplayBuffer,
    # Parameters for HER
    replay_buffer_kwargs=dict(
        n_sampled_goal=4,
        goal_selection_strategy=goal_selection_strategy,
        online_sampling=online_sampling
    ),
    verbose=1,
)
# Train the model and then save it
model_td3.learn(total_timesteps=n_timesteps, log_interval=log_interval)

model_td3.save("ddpg_fetch_reach_her")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to ./fetchreachHerGraphs/SAC_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 50       |
|    ep_rew_mean     | -50      |
|    success rate    | 0.0      |
| time/              |          |
|    episodes        | 4        |
|    fps             | 438      |
|    time_elapsed    | 0        |
|    total timesteps | 200      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 50       |
|    ep_rew_mean     | -49.9    |
|    success rate    | 0.0      |
| time/              |          |
|    episodes        | 8        |
|    fps             | 460      |
|    time_elapsed    | 0        |
|    total timesteps | 400      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 50       |
|    ep_rew_mean 

In [9]:
model = model_class_ddpg.load('ddpg_fetch_reach_her', env=env)


for i_episode in range(20):
    obs = env.reset()
    for t in range(100):
        env.render()
        action, _states = model.predict(obs)
        obs, rewards, dones, info = env.step(action)
        if dones:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Creating window glfw
Episode finished after 50 timesteps
Episode finished after 50 timesteps
Episode finished after 50 timesteps
Episode finished after 50 timesteps


SystemExit: 0

/home/pasc/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
